In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import random

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor, hooks
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
import os
import numpy as np
import json

def get_semseg_dataset_dicts(root_dir, dataset_name, mode):
    '''
    filelist is expected to be ADE20K style
    
    '''
    listpath = os.path.join(root_dir, dataset_name, 
                            "training_customdataset.odgt" if mode == 'train' else "validation_customdataset.odgt")
    filelist = []
    with open(listpath) as f:
        lines = f.readlines()
        for line in lines:
            img_s_id = line.find('fpath_img') + 13 # fpath_img": " = 13 characters
            img_e_id = line[img_s_id:].find('"') + img_s_id
            lbl_s_id = line.find('fpath_segm') + 14 # fpath_segm": " = 14 characters
            lbl_e_id = line[lbl_s_id:].find('"') + lbl_s_id
            data = {'filename':line[img_s_id : img_e_id], 
                    'labelname':line[lbl_s_id : lbl_e_id]}
#             print(data)
            filelist.append(data)

    dataset_dicts = []

    for i, datapaths in enumerate(filelist):
        record = {}
        
        filename = os.path.join(root_dir, datapaths["filename"])
        labelname = os.path.join(root_dir, datapaths["labelname"])
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["sem_seg_file_name"] = labelname
        record["image_id"] = i
        record["height"] = height
        record["width"] = width
 
        dataset_dicts.append(record)
    return dataset_dicts


In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog
for mode in ["train", "val"]:
    root_dir = '../data/detectron2'
    dataset_name = 'training_data'
    DatasetCatalog.register("semseg_" + mode, lambda d=mode:get_semseg_dataset_dicts(root_dir, dataset_name, mode))
    MetadataCatalog.get("semseg_" + mode).set(stuff_classes=["others", "target"], 
                                             stuff_colors=[(120,120,120),(255,0,0)])
customdata_metadata = MetadataCatalog.get("semseg_train")
customdata_metadata_val = MetadataCatalog.get("semseg_val")
print(customdata_metadata, customdata_metadata_val)

In [ ]:
dataset_dicts = get_semseg_dataset_dicts(root_dir, dataset_name, 'train')

In [ ]:
def visualize_dataset(dataset_dicts):
    for d in random.sample(dataset_dicts, 3):
        img = cv2.imread(d["file_name"])
        visualizer = Visualizer(img[:, :, ::-1], metadata=customdata_metadata, scale=0.5)
        vis = visualizer.draw_dataset_dict(d)
        im = vis.get_image()[:, :, ::-1]
        im = np.array(im, dtype=np.uint8)
        im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
        plt.imshow(im)
        plt.show()
visualize_dataset(dataset_dicts)

In [ ]:
from detectron2.modeling import GeneralizedRCNNWithTTA
class Trainer(DefaultTrainer):
    ignore_in_eval = None
    
    def __init__(self, cfg, ignore_in_eval=None):
        super(Trainer, self).__init__(cfg)
        Trainer.ignore_in_eval = ignore_in_eval
        
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        print('Evaluation with', dataset_name, cls.ignore_in_eval)
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return SemSegEvaluator(
            dataset_name,
            distributed=True,
            num_classes=cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES,
            ignore_label=cfg.MODEL.SEM_SEG_HEAD.IGNORE_VALUE if cls.ignore_in_eval is None else cls.ignore_in_eval,
            output_dir=output_folder,
        )

    @classmethod
    def test_with_TTA(cls, cfg, model):
        logger = logging.getLogger("detectron2.trainer")
        # In the end of training, run an evaluation with TTA
        # Only support some R-CNN models.
        logger.info("Running inference with test-time augmentation ...")
        model = GeneralizedRCNNWithTTA(cfg, model)
        evaluators = [
            cls.build_evaluator(
                cfg, name, output_folder=os.path.join(cfg.OUTPUT_DIR, "inference_TTA")
            )
            for name in cfg.DATASETS.TEST
        ]
        res = cls.test(cfg, model, evaluators)
        res = OrderedDict({k + "_TTA": v for k, v in res.items()})
        return res

In [ ]:
from detectron2.config import get_cfg
from detectron2.config import CfgNode as CN
from detectron2.evaluation import SemSegEvaluator, DatasetEvaluators

cfg = get_cfg()
# cfg.merge_from_file('configs/Misc/semantic_R_50_FPN_1x.yaml')
cfg.merge_from_file('configs/Misc/semantic_R_101_FPN_1x.yaml')
cfg.DATASETS.TRAIN = ("semseg_train",)
cfg.DATASETS.TEST = ("semseg_val",)
cfg.DATALOADER.NUM_WORKERS = 16
cfg.INPUT.MAX_SIZE_TRAIN = 2000
cfg.INPUT.CROP = CN({"ENABLED": True})
cfg.INPUT.CROP.TYPE = "relative_range"
cfg.INPUT.CROP.SIZE = [0.9, 0.9]
cfg.INPUT.MASK_FORMAT = "bitmask" # ???
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("Misc/semantic_R_50_FPN_1x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0005  # pick a good LR
cfg.SOLVER.MAX_ITER = 100
cfg.TEST.EVAL_PERIOD = 50
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 2
cfg.MODEL.SEM_SEG_HEAD.IGNORE_VALUE = -1
cfg.OUTPUT_DIR = "./output/"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg) 
trainer = Trainer(cfg, ignore_in_eval=0)
trainer.resume_or_load(resume=False)

# trainer.register_hooks(
#             [hooks.EvalHook(0, lambda: trainer.test_with_TTA(cfg, trainer.model))]
#         )
trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
# cfg.DATASETS.TEST = ("custom_val", )
predictor = DefaultPredictor(cfg)

In [ ]:
root_dir = '../data/detectron2'
dataset_name = 'test_512px'
dataset_dicts_test = get_semseg_dataset_dicts(root_dir, dataset_name, 'val')
DatasetCatalog.register("semseg_test", lambda d=mode:get_semseg_dataset_dicts(root_dir, dataset_name, 'val'))
MetadataCatalog.get("semseg_test").set(stuff_classes=["others", "target"], 
                                         stuff_colors=[(120,120,120),(255,0,0)])
customdata_metadata_test = MetadataCatalog.get("semseg_test")

In [ ]:
import glob 

os.makedirs('./results', exist_ok=True)

for i, d in enumerate(dataset_dicts_test):
    print(i, d['file_name'])
    im = cv2.imread(d["file_name"])
    im = np.array(im, dtype=np.uint8)
#     plt.figure(figsize=(15,15))
#     plt.imshow(cv2.cvtColor(im, cv2.COLOR_RGB2BGR))
#     plt.show()

    outputs = predictor(im)
    out = np.array(outputs["sem_seg"].argmax(dim=0).to('cpu')*255, dtype=np.uint8)
    cv2.imwrite('./results/out{}_base.png'.format(i), im)
#     cv2.imwrite('./results/out{}_mask.png'.format(i), out)
    v = Visualizer(im[:, :, ::-1],
                   metadata=customdata_metadata_test, 
                   scale=1.0, 
#                    instance_mode=ColorMode.SEGMENTATION   # remove the colors of unsegmented pixels
    )

    vis = v.draw_sem_seg(outputs["sem_seg"].argmax(dim=0).to("cpu"), area_threshold=0, alpha=0.5)
    im = vis.get_image()[:, :, ::-1]
    im = np.array(im, dtype=np.uint8)
    cv2.imwrite('./results/out{}.png'.format(i), im)
    im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
    plt.figure(figsize=(8,8))
    plt.imshow(im)
    plt.show()
#     cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
from detectron2.evaluation import SemSegEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.checkpoint import DetectionCheckpointer

evaluator = SemSegEvaluator("semseg_test", distributed=True, num_classes=2, 
                            ignore_label=0, output_dir="./output/")
# model = trainer.build_model(cfg)
# DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
#             cfg.MODEL.WEIGHTS)
trainer.test(cfg, trainer.model, evaluator)
# trainer.test(cfg, model, evaluator)